# Optimisation d'un modèle XGBoost pour la Prédiction de Risque Cardiaque

Ce notebook a pour objectif de :
1.  Charger les données sur le risque cardiaque.
2.  Préparer les données pour l'entraînement.
3.  Utiliser `GridSearchCV` pour trouver les meilleurs hyperparamètres pour un classificateur XGBoost.
4.  Évaluer le modèle final sur l'ensemble de test.

## XGbost

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from xgboost import XGBClassifier

import time

# --- 1. Charger les données ---
print("Chargement des données...")
try:
    df = pd.read_csv('data/donnees_traitees_binaire.csv')
    df.columns = df.columns.str.strip()
    print("Données chargées.")
except FileNotFoundError:
    print("ERREUR: Fichier 'data/donnees_traitees_binaire.csv' introuvable.")
    exit()

# --- 2. Séparer X (features) et y (cible) ---
target_col = 'Heart Attack Risk (Binary)'
y = df[target_col].round(0).astype(int)
X = df.drop(columns=[target_col])

# --- 3. Pré-traitement (One-Hot Encoding) sur X ---
print("Début du pré-traitement...")
categorical_vars = [
    "Diabetes", "Family History", "Smoking", "Obesity", "Alcohol Consumption",
    "Diet", "Previous Heart Problems", "Medication Use", "Gender",
    "Stress Level", "Physical Activity Days Per Week"
]
for col in X.columns:
    if col in categorical_vars:
        X[col] = X[col].round(0).astype(int).astype("category")

X_encoded = pd.get_dummies(X, drop_first=True)
print(f"Pré-traitement terminé. Nombre de features : {X_encoded.shape[1]}")

# --- 4. Split Train/Test ---
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y # J'ai remis random_state=42 pour la cohérence
)
print(f"\nTaille X_train : {X_train.shape}")
print(f"Distribution de y_train : {y_train.value_counts().to_dict()}")

# --- 5. CALCUL DU POIDS ---
# Calcule le ratio pour 'scale_pos_weight'
counts = y_train.value_counts()
scale_pos_weight = counts[0] / counts[1]
print(f"\nCalcul de 'scale_pos_weight' pour gérer le déséquilibre : {scale_pos_weight:.2f}")

## --- 6. GridSearchCV (avec 'scale_pos_weight') ---
print("\nLancement de GridSearchCV (avec scale_pos_weight)...")
start_time = time.time()

xgb = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=scale_pos_weight
)

param_grid = {
    'max_depth': [2, 3], 
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.05], 
    'subsample': [0.7, 0.8], 
    'colsample_bytree': [0.7, 0.8],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [1, 2, 5, 10]
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1_macro', 
    cv=3,
    verbose=1,
    n_jobs=1  
)

print("DÉBUT DE L'ENTRAÎNEMENT (sur les données originales)...")
grid_search.fit(X_train, y_train) 
end_time = time.time()
print(f"GridSearchCV terminé. (Durée: {end_time - start_time:.2f} secondes)")

# --- 7. Évaluation du Meilleur Modèle ---
print("\n--- Évaluation du Meilleur Modèle ---")
best_model = grid_search.best_estimator_

print(f"Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)
print(f"Meilleur score F1-Macro (pendant la validation croisée) : {grid_search.best_score_:.4f}")

# --- Évaluation TEST ---
y_pred_test = best_model.predict(X_test)

print(f"\nRapport de classification TEST (Seuil 0.5 par défaut) :")
print(classification_report(y_test, y_pred_test))
print("Matrice de confusion TEST :")
print(confusion_matrix(y_test, y_pred_test))

# --- Évaluation TRAIN (Pour vérifier l'overfitting) ---

y_pred_train = best_model.predict(X_train)

print(f"\nRapport de classification TRAIN (Seuil 0.5 par défaut) :")
# MODIFICATION : On compare à y_train 
print(classification_report(y_train, y_pred_train))
print(" Matrice de confusion TRAIN :")
print(confusion_matrix(y_train, y_pred_train))

# --- Comparaison des scores ---

f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')

print("\nComparaison des scores (F1-Macro) :")
print(f"F1-score (Macro) - TRAIN : {f1_train:.4f}")
print(f"F1-score (Macro) - TEST  : {f1_test:.4f}")

if f1_train > (f1_test + 0.1):
    print("Avertissement : Overfitting possible (écart > 10% entre train et test).")
else:
    print("Le modèle semble bien généraliser.")

Chargement des données...
Données chargées.
Début du pré-traitement...
Pré-traitement terminé. Nombre de features : 34

Taille X_train : (7501, 34)
Distribution de y_train : {0: 4836, 1: 2665}

Calcul de 'scale_pos_weight' pour gérer le déséquilibre : 1.81

Lancement de GridSearchCV (avec scale_pos_weight)...
DÉBUT DE L'ENTRAÎNEMENT (sur les données originales)...
Fitting 3 folds for each of 384 candidates, totalling 1152 fits


c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:42:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:42:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:42:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:42:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update

GridSearchCV terminé. (Durée: 102.19 secondes)

--- Évaluation du Meilleur Modèle ---
Meilleurs hyperparamètres trouvés :
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7}
Meilleur score F1-Macro (pendant la validation croisée) : 0.5157

Rapport de classification TEST (Seuil 0.5 par défaut) :
              precision    recall  f1-score   support

           0       0.65      0.50      0.56      1210
           1       0.36      0.52      0.43       666

    accuracy                           0.50      1876
   macro avg       0.51      0.51      0.50      1876
weighted avg       0.55      0.50      0.52      1876

Matrice de confusion TEST :
[[600 610]
 [319 347]]

Rapport de classification TRAIN (Seuil 0.5 par défaut) :
              precision    recall  f1-score   support

           0       0.78      0.60      0.68      4836
           1       0.49      0.69      0.57      2665

    accuracy       

c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:44:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## code avec donnees synthetiques pour diminuer leffet du desiquilibre

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
import time

# --- 1. Charger les données ---
print("Chargement des données...")
try:
    df = pd.read_csv('data/donnees_traitees_binaire.csv')
    df.columns = df.columns.str.strip()
    print("Données chargées.")
except FileNotFoundError:
    print("ERREUR: Fichier 'data/donnees_traitees_binaire.csv' introuvable.")
    exit()

# --- 2. Séparer X (features) et y (cible) ---
target_col = 'Heart Attack Risk (Binary)'
y = df[target_col].round(0).astype(int)
X = df.drop(columns=[target_col])

# --- 3. Pré-traitement (One-Hot Encoding) sur X ---
print("Début du pré-traitement...")
categorical_vars = [
    "Diabetes", "Family History", "Smoking", "Obesity", "Alcohol Consumption",
    "Diet", "Previous Heart Problems", "Medication Use", "Gender",
    "Stress Level", "Physical Activity Days Per Week"
]
for col in X.columns:
    if col in categorical_vars:
        X[col] = X[col].round(0).astype(int).astype("category")

X_encoded = pd.get_dummies(X, drop_first=True)
print(f"Pré-traitement terminé. Nombre de features : {X_encoded.shape[1]}")

# --- 4. Split Train/Test ---
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=2, stratify=y
)
print(f"\nTaille X_train avant SMOTETomek : {X_train.shape}")
print(f"Taille y_train avant SMOTETomek : {y_train.value_counts().to_dict()}")

# --- 5. Rééchantillonnage (SMOTETomek) ---
print("\nApplication de SMOTETomek sur les données d'entraînement...")
smote_tomek = SMOTETomek(random_state=42)
X_train_res, y_train_res = smote_tomek.fit_resample(X_train, y_train)
print(f"Taille X_train après SMOTETomek : {X_train_res.shape}")
print(f"Taille y_train après SMOTETomek : {y_train_res.value_counts().to_dict()}")

# --- 6. GridSearchCV (avec une GRILLE PLUS STRICTE) ---
print("\nLancement de GridSearchCV (avec grille de régularisation)...")
start_time = time.time()

xgb = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Nouvelle grille conçue pour RÉDUIRE l'overfitting
param_grid = {
    'max_depth': [2, 3], 
    'n_estimators': [20, 50],
    'learning_rate': [0.01, 0.05], 
    'subsample': [0.7, 0.8], 
    'colsample_bytree': [0.7, 0.8],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [1, 2, 5, 10]
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1_macro', 
    cv=3,
    verbose=1,
    n_jobs=-1
)


print("DÉBUT DE L'ENTRAÎNEMENT (GRIDSEARCH.FIT)...")
grid_search.fit(X_train_res, y_train_res)
end_time = time.time()
print(f"GridSearchCV terminé. (Durée: {end_time - start_time:.2f} secondes)")



# --- 7. Évaluation du Meilleur Modèle ---
print("\n--- Évaluation du Meilleur Modèle ---")

best_model = grid_search.best_estimator_

print(f"Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)
print(f"Meilleur score F1-Macro (pendant la validation croisée) : {grid_search.best_score_:.4f}")

# --- Évaluation TEST  ---
y_pred_test = best_model.predict(X_test)

print(f"\nRapport de classification TEST (Seuil 0.5 par défaut) :")
print(classification_report(y_test, y_pred_test))
print("Matrice de confusion TEST :")
print(confusion_matrix(y_test, y_pred_test))

# --- Évaluation TRAIN (Pour vérifier l'overfitting) ---
y_pred_train = best_model.predict(X_train_res)

print(f"\nRapport de classification TRAIN (Seuil 0.5 par défaut) :")
print(classification_report(y_train_res, y_pred_train))
print(" Matrice de confusion TRAIN :")
print(confusion_matrix(y_train_res, y_pred_train))

# --- Comparaison des scores ---
f1_train = f1_score(y_train_res, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')

print("\nComparaison des scores (F1-Macro) :")
print(f"F1-score (Macro) - TRAIN : {f1_train:.4f}")
print(f"F1-score (Macro) - TEST  : {f1_test:.4f}")

if f1_train > (f1_test + 0.1):
    print("Avertissement : Overfitting possible (écart > 10% entre train et test).")
else:
    print("Le modèle semble bien généraliser.")

Chargement des données...
Données chargées.
Début du pré-traitement...
Pré-traitement terminé. Nombre de features : 34

Taille X_train avant SMOTETomek : (7501, 34)
Taille y_train avant SMOTETomek : {0: 4836, 1: 2665}

Application de SMOTETomek sur les données d'entraînement...
Taille X_train après SMOTETomek : (9182, 34)
Taille y_train après SMOTETomek : {1: 4591, 0: 4591}

Lancement de GridSearchCV (avec grille de régularisation)...
DÉBUT DE L'ENTRAÎNEMENT (GRIDSEARCH.FIT)...
Fitting 3 folds for each of 384 candidates, totalling 1152 fits
GridSearchCV terminé. (Durée: 11.65 secondes)

--- Évaluation du Meilleur Modèle ---
Meilleurs hyperparamètres trouvés :
{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 20, 'reg_alpha': 0.1, 'reg_lambda': 2, 'subsample': 0.8}
Meilleur score F1-Macro (pendant la validation croisée) : 0.6088

Rapport de classification TEST (Seuil 0.5 par défaut) :
              precision    recall  f1-score   support

           0     

c:\Users\Jamila\anaconda3\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:21:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
